In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv("laptop_price.csv", encoding ="latin-1")
df

In [ ]:
df = df.drop("Product", axis = 1)
df

In [ ]:
df = df.join(pd.get_dummies(df['Company'], dtype=int))
df = df.drop("Company", axis=1)

df = df.join(pd.get_dummies(df['TypeName'], dtype=int))
df = df.drop("TypeName", axis=1)

df

In [777]:
df["ScreenResolution"] = df.ScreenResolution.str.split(" ").apply(lambda x: x[-1])
df["Screen Width"] = df.ScreenResolution.str.split("x").apply(lambda x: x[0])
df["Screen Height"] = df.ScreenResolution.str.split("x").apply(lambda x: x[1])

df = df.drop("ScreenResolution", axis=1)

df["CPU Brand"] = df.Cpu.str.split(" ").apply(lambda x: x[0])
df["CPU Frequency"] = df.Cpu.str.split(" ").apply(lambda x: x[-1])

df = df.drop("Cpu", axis=1)

df["CPU Frequency"] = df["CPU Frequency"].str[:-3]

df["Ram"] = df["Ram"].str[:-2]

df["Ram"] = df["Ram"].astype("int")
df["CPU Frequency"] = df["CPU Frequency"].astype("float")

df["Screen Width"] = df["Screen Width"].astype("int")
df["Screen Height"] = df["Screen Height"].astype("int")

df["Memory Amount"] = df.Memory.str.split(" ").apply(lambda x: x[0])
df["Memory Type"] = df.Memory.str.split(" ").apply(lambda x: x[1])

def turn_memory_into_MB(value):
    if "GB" in value:
        return float(value[:value.find("GB")]) * 1000
    elif "TB" in value:
        return float(value[:value.find("TB")]) * 1000000

df["Memory Amount"] = df["Memory Amount"].apply(turn_memory_into_MB)

df = df.drop("Memory", axis=1)
df["Weight"] = df["Weight"].str[:-2]
df["Weight"] = df["Weight"].astype("float")

df["GPU Brand"] = df.Gpu.str.split(" ").apply(lambda x: x[0])
df = df.drop("Gpu", axis = 1)


df = df.join(pd.get_dummies(df['OpSys'], dtype=int))
df = df.drop("OpSys", axis=1)




In [778]:
cpu_categories = pd.get_dummies(df["CPU Brand"], dtype = int)
cpu_categories.columns = [col+"_CPU" for col in cpu_categories.columns]

df = df.join(cpu_categories)
df = df.drop("CPU Brand", axis = 1)


gpu_categories = pd.get_dummies(df["GPU Brand"], dtype = int)
gpu_categories.columns = [col+"_GPU" for col in gpu_categories.columns]

df = df.join(gpu_categories)
df = df.drop("GPU Brand", axis = 1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20,15))

numeric_df = df.select_dtypes(include='number')

sns.heatmap(numeric_df.corr(), annot=True, cmap="YlGnBu")

plt.show()


In [ ]:

numeric_df = df.select_dtypes(include='number')
target_correlations = numeric_df.corr()['Price_euros'].apply(abs).sort_values()
selected_features = list(target_correlations[-20:].index)


target_correlations

In [ ]:
selected_features = target_correlations[-21:].index
selected_features = list(selected_features)
selected_features

In [ ]:
limited_df = df[selected_features]
limited_df

In [ ]:
plt.figure(figsize=(20,15))

numeric_df = limited_df.select_dtypes(include='number')

sns.heatmap(numeric_df.corr(), annot=True, cmap="YlGnBu")

plt.show()

In [784]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X,y = limited_df.drop("Price_euros", axis=1), limited_df["Price_euros"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

X_test_orig = X_test.copy()


Scaler = StandardScaler()

X_train = Scaler.fit_transform(X_train)
X_test = Scaler.transform(X_test)

y_pred = forest.predict(X_test)


In [ ]:
forest = RandomForestRegressor()
forest.fit(X_train, y_train)


In [ ]:
forest.score(X_test, y_test)

In [ ]:
plt.figure(figsize=(12,9))
plt.scatter(y_pred, y_test)

plt.plot(range(0,6000), range(0,6000), c = "red")

In [ ]:
X_new_scaled = Scaler.transform(X_test_orig.iloc[[0]])

forest.predict(X_new_scaled)

In [ ]:
y_test.iloc[0]

In [790]:
def predict_laptop_price(user_input_dict):
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity

    # Convert dict to DataFrame
    input_df = pd.DataFrame([user_input_dict])

    # Add missing columns
    for col in model_features:
        if col not in input_df.columns:
            input_df[col] = 0

    # Reorder columns
    input_df = input_df[model_features]

    # Scale input
    input_scaled = Scaler.transform(input_df)

    # Predict price
    predicted_price = forest.predict(input_scaled)[0]
    fluctuation = 0.1 * predicted_price  # ±10% range

    # Output predicted range
    print("====================================")
    print(f"💰 Estimated Price: €{predicted_price:.2f}")
    print(f"📊 Likely Range: €{predicted_price - fluctuation:.2f} to €{predicted_price + fluctuation:.2f}")
    print("====================================\n")

    # Show similar examples from dataset
    print("🖥️ Similar laptops from dataset:")

    # Scale entire dataset
    X_scaled_full = Scaler.transform(X)

    # Cosine similarity to find similar rows
    similarities = cosine_similarity(input_scaled, X_scaled_full)[0]
    top_indices = similarities.argsort()[-3:][::-1]  # top 3 similar

    similar_laptops = limited_df.iloc[top_indices]

    for idx, row in similar_laptops.iterrows():
        # Determine GPU brand text
        gpu_brand = []
        if row['Nvidia_GPU'] == 1:
            gpu_brand.append("Nvidia")
        if row['Intel_GPU'] == 1:
            gpu_brand.append("Intel")
        if row['AMD_GPU'] == 1:
            gpu_brand.append("AMD")
        gpu_brand_text = ", ".join(gpu_brand) if gpu_brand else "Unknown"

        # Additional flags for notebook type
        types = []
        for t in ['Notebook', 'Gaming', 'Ultrabook', 'Workstation']:
            if row.get(t, 0) == 1:
                types.append(t)
        type_text = ", ".join(types) if types else "Other"

        print(f"- Price: €{row['Price_euros']:.2f}, RAM: {row['Ram']} GB, CPU Freq: {row['CPU Frequency']} GHz, Weight: {row['Weight']} kg, GPU: {gpu_brand_text}, Type: {type_text}")

    return predicted_price



In [791]:
def get_user_input():
    print("🔧 Please enter the following laptop specs:")
    # RAM
    ram = int(input("Enter RAM (GB) [e.g., 4, 8, 16, 32]: "))
    # Weight
    weight = float(input("Enter Weight (kg) [e.g., 1.2, 1.5, 1.8, 2.0]: "))
    # CPU Frequency
    cpu_freq = float(input("Enter CPU Frequency (GHz) [e.g., 2.0, 2.5, 3.2]: "))

    # Screen Resolution examples for width and height
    print("Common screen resolutions:")
    print("1: 1366 x 768")
    print("2: 1920 x 1080")
    print("3: 2560 x 1440")
    print("4: 3840 x 2160 (4K)")

    screen_choice = input("Choose screen resolution by number or enter custom width height separated by comma (e.g. 1920,1080): ")

    if ',' in screen_choice:
        width, height = map(int, screen_choice.split(','))
    else:
        choices = {
            '1': (1366, 768),
            '2': (1920, 1080),
            '3': (2560, 1440),
            '4': (3840, 2160)
        }
        width, height = choices.get(screen_choice, (1920,1080))  # default to 1920x1080

    # Category flags
    notebook = int(input("Is it a Notebook? (1 for Yes, 0 for No): "))
    gaming = int(input("Is it a Gaming laptop? (1 for Yes, 0 for No): "))
    ultrabook = int(input("Is it an Ultrabook? (1 for Yes, 0 for No): "))
    workstation = int(input("Is it a Workstation? (1 for Yes, 0 for No): "))

    # Brands (update this list based on your dataset's brand columns)
    brands = ['Acer', 'Razer', 'Dell', 'HP', 'Lenovo', 'Asus', 'Apple', 'MSI']
    print("Available Brands:")
    for i, b in enumerate(brands):
        print(f"{i+1}: {b}")
    brand_choice = int(input(f"Choose brand by number (1-{len(brands)}): "))
    brand_dict = {b:0 for b in brands}
    if 1 <= brand_choice <= len(brands):
        brand_dict[brands[brand_choice-1]] = 1
    else:
        print("Invalid brand choice, defaulting all to 0")

    # OS options
    os_options = ['Windows 7', 'Linux', 'No OS', 'Windows 10', 'Windows 11']
    print("Operating Systems:")
    for i, os in enumerate(os_options):
        print(f"{i+1}: {os}")
    os_choice = int(input(f"Choose OS by number (1-{len(os_options)}): "))
    os_dict = {os_name: 0 for os_name in ['Windows 7', 'Linux', 'No OS']}
    if os_choice == 1:
        os_dict['Windows 7'] = 1
    elif os_choice == 2:
        os_dict['Linux'] = 1
    elif os_choice == 3:
        os_dict['No OS'] = 1
    # If Windows 10 or 11, leave all 0 (assuming your model columns match)

    # CPU Brand
    cpu_brands = ['AMD_CPU', 'Intel_CPU']
    print("CPU Brands:")
    for i, c in enumerate(cpu_brands):
        print(f"{i+1}: {c[:-4]}")  # Show without _CPU suffix
    cpu_choice = int(input(f"Choose CPU brand (1-{len(cpu_brands)}): "))
    cpu_dict = {c: 0 for c in cpu_brands}
    if 1 <= cpu_choice <= len(cpu_brands):
        cpu_dict[cpu_brands[cpu_choice-1]] = 1

    # GPU Brand
    gpu_brands = ['Intel_GPU', 'AMD_GPU', 'Nvidia_GPU']
    print("GPU Brands:")
    for i, g in enumerate(gpu_brands):
        print(f"{i+1}: {g[:-4]}")
    gpu_choice = int(input(f"Choose GPU brand (1-{len(gpu_brands)}): "))
    gpu_dict = {g: 0 for g in gpu_brands}
    if 1 <= gpu_choice <= len(gpu_brands):
        gpu_dict[gpu_brands[gpu_choice-1]] = 1

    # Build final dictionary with all required features
    user_input = {
        'Ram': ram,
        'Weight': weight,
        'CPU Frequency': cpu_freq,
        'Screen Width': width,
        'Screen Height': height,
        'Notebook': notebook,
        'Gaming': gaming,
        'Ultrabook': ultrabook,
        'Workstation': workstation,
        **brand_dict,
        **os_dict,
        **cpu_dict,
        **gpu_dict
    }

    return user_input



In [793]:
user_input_dict = get_user_input()
predict_laptop_price(user_input_dict)

🔧 Please enter the following laptop specs:
Enter RAM (GB) [e.g., 4, 8, 16, 32]: 16
Enter Weight (kg) [e.g., 1.2, 1.5, 1.8, 2.0]: 1.5
Enter CPU Frequency (GHz) [e.g., 2.0, 2.5, 3.2]: 2.5
Common screen resolutions:
1: 1366 x 768
2: 1920 x 1080
3: 2560 x 1440
4: 3840 x 2160 (4K)
Choose screen resolution by number or enter custom width height separated by comma (e.g. 1920,1080): 1920,1080
Is it a Notebook? (1 for Yes, 0 for No): 1
Is it a Gaming laptop? (1 for Yes, 0 for No): 0
Is it an Ultrabook? (1 for Yes, 0 for No): 0
Is it a Workstation? (1 for Yes, 0 for No): 0
Available Brands:
1: Acer
2: Razer
3: Dell
4: HP
5: Lenovo
6: Asus
7: Apple
8: MSI
Choose brand by number (1-8): 6
Operating Systems:
1: Windows 7
2: Linux
3: No OS
4: Windows 10
5: Windows 11
Choose OS by number (1-5): 5
CPU Brands:
1: AMD
2: Intel
Choose CPU brand (1-2): 2
GPU Brands:
1: Intel
2: AMD
3: Nvidia
Choose GPU brand (1-3): 1
💰 Estimated Price: €1402.39
📊 Likely Range: €1262.15 to €1542.63

🖥️ Similar laptops from 

np.float64(1402.387666666667)